# Explore: Amtrak Services

## Intercity Passenger Rail Service Station Performance Metrics

The Amtrak [network](https://www.amtrak.com/content/dam/projects/dotcom/english/public/documents/Maps/Amtrak-System-Map-020923.pdf)
is a passenger rail service that provides intercity rail service in the
continental United States and to select Canadian cities. The network is operated by the
[National Railroad Passenger Corporation](https://railroads.dot.gov/passenger-rail/amtrak/amtrak),
a federally chartered for-profit corporation that receives some state funding and covers its
operating costs by selling tickets and providing other services.

This notebook commences exploration of the augmented quarterly
[Amtrak](https://www.amtrak.com/home.html) station performance metrics. The goal is to better
understand Amtrak's three service lines and identify potential areas for further analysis.

### Variable names

A number of variable names in this project leverage the following abbreviations. The naming
strategy is to strike a balance between brevity and readability:

* `amtk`: Amtrak (reporting mark)
* `chrt`: chart
* `cols`: columns
* `const`: constant
* `cwd`: current working directory
* `eb`: eastbound direction of travel
* `lm`: linear model
* `mi`: miles
* `mm`: minutes (ISO 8601)
* `nb`: northbound direction of travel
* `psgr`: passenger
* `qtr`: quarter
* `rte`: route
* `sb`: southbound direction of travel
* `stats`: summary statistics
* `stn`: station
* `stns`: stations
* `svc`: service
* `trn`: train
* `wb`: westbound direction of travel

In [ ]:
import numpy as np
import pandas as pd
import pathlib as pl
import tomllib as tl

import fra_amtrak.amtk_detrain as detrn
import fra_amtrak.amtk_frame as frm
import fra_amtrak.amtk_network as ntwk
import fra_amtrak.chart_bar as bar
import fra_amtrak.chart_box_preagg as boxp
import fra_amtrak.chart_hist as hst
import fra_amtrak.chart_title as ttl

## 1.0 Read files

### 1.1 Resolve paths

In [ ]:
parent_path = pl.Path.cwd()  # current working directory
parent_path

### 1.2 Load constants

Load a companion [TOML](https://toml.io/en/) file containing constants.

In [ ]:
filepath = parent_path.joinpath("notebook.toml")
with open(filepath, "rb") as file_obj:
    const = json.load(file_obj)

# Access constants
AGG = const["agg"]
CHRT_BAR = const["chart"]["bar"]
COLORS = const["colors"]
COLS = const["columns"]
SVC_LINES = const["service_lines"]


### 1.3 Retrieve performance data

In [ ]:
filepath = parent_path.joinpath("data", "processed", "station_performance_metrics-v1p2.csv")
network = pd.read_csv(
    filepath, dtype={"Address 02": "str", "ZIP Code": "str"}, low_memory=False
)  # avoid DtypeWarning
network.shape

## 2.0 Amtrak service lines

Provide service line summary statistics covering all available fiscal years and quarters.

In [ ]:
svc_line_stats = detrn.get_sum_stats_by_group(
    network, COLS["svc_line"], AGG["columns"], AGG["funcs"]
)
svc_line_stats

## 3.0 Northeast Corridor (NEC)

In [ ]:
nec = ntwk.by_service_line(network, SVC_LINES["nec"])
nec.shape

### 3.1 NEC: on-time performance metrics (entire period)

NEC station performance data is a compilation of quarterly metrics that focus on late
detraining passengers. Detraining assengers are considered on-time if they arrive at their
destination no later than fifteen (`15`) minutes after their scheduled arrival time. All other
detraining passengers are considered late.

In [ ]:
# Total train arrivals
nec_trn_arrivals = nec.shape[0]

# Detraining totals
nec_detrn = nec[COLS["total_detrn"]].sum()
nec_detrn_late = nec[COLS["late_detrn"]].sum()
nec_detrn_on_time = nec_detrn - nec_detrn_late

print(
    f"Train Arrivals: {nec_trn_arrivals}",
    f"Total Detraining Customers: {nec_detrn}",
    f"Late Detraining Customers: {nec_detrn_late}",
    f"On-Time Detraining Customers: {nec_detrn_on_time}",
    sep="\n",
)

# Compute summary statistics
nec_stats = detrn.get_sum_stats(nec, AGG["columns"], AGG["funcs"])
nec_stats

### 3.2 NEC: mean late arrival times summary statistics [1 pt]

Review the central tendency, dispersion, and shape for the mean late arrival times of Northeast Corridor trains. Call the custom function named `frm.describe_numeric_column()` to return a dictionary of summary statistics.

In [ ]:
# Drop missing values
nec_avg_mm_late = nec[COLS["late_detrn_avg_mm_late"]].reset_index(drop=True)

# Call the custom frm.describe_numeric_column() function again
nec_avg_mm_late_describe = frm.describe_numeric_column(nec_avg_mm_late)
nec_avg_mm_late_describe

In [ ]:
#hidden tests are within this cell

The skewness and kurtosis values returned suggest that the distribution of mean late arrival times of NEC trains are positively skewed and features a sharper peak and heavier right tail than a normal distribution. Let's confirm this visually by generating a histogram.

### 3.3 NEC: visualize distribution of mean late arrival times

Visualize mean late arrival times for the entire period. The data is binned prior to plotting.

#### 3.3.1 Create the chart data

In [ ]:
# Convert to DataFrame
nec_avg_mm_late = nec_avg_mm_late.to_frame(name=COLS["avg_mm_late"])

# Get mean and standard deviation
mu = nec_avg_mm_late_describe["center"]["mean"]
sigma = nec_avg_mm_late_describe["spread"]["std"]

# Get max value (for x-axis ticks); pad max value for chart display
max_val = nec_avg_mm_late_describe["position"]["max"]
max_val_ceil = (np.ceil(max_val / 10) * 10).astype(int)

# Create bins
nec_mm_late, bins, num_bins, bin_width = frm.create_bins(nec_avg_mm_late, COLS["avg_mm_late"], 15)

# Bin the data
chrt_data = frm.bin_data(nec_mm_late, COLS["avg_mm_late"], bins)
# chrt_data

#### 3.3.2 Generate the histogram

In [ ]:
# Chart title
title_txt = f"Amtrak {SVC_LINES['nec']} (NEC) Late Detraining Passengers"
title = ttl.format_title(nec_stats, title_txt)

# Tooltips
tooltip_config = [
    {"shorthand": "bin_center:Q", "title": "Average Minutes Late", "format": None},
    {"shorthand": "count:Q", "title": "Late Arrivals Count", "format": None},
]

# Create and display the histogram
chart = hst.create_histogram(
    frame=chrt_data,
    x_shorthand="bin_center:Q",
    x_title="Average Minutes Late",
    y_shorthand="count:Q",
    y_title="Late Arrivals Count",
    y_stack=False,
    line_shorthand="Avg Min Late:Q",
    mu=mu,
    sigma=sigma,
    num_bins=num_bins,
    bin_width=bin_width,
    x_tick_count_max=max_val_ceil,
    bar_color=COLORS["amtk_blue"],
    mu_color=COLORS["amtk_red"],
    sigma_color=COLORS["anth_gray"],
    tooltip_config=tooltip_config,
    title=title,
)
chart.display()


### 3.4 NEC: on-time performance metrics (by fiscal year and quarter)

Compute OTP summary statistics per fiscal year and quarter. Add quarterly train arrival metrics to
the `DataFrame` named `nec_qtr_stats`.

In [ ]:
# Get quarterly stats
nec_qtr_stats = detrn.get_sum_stats_by_group(
    nec,
    [COLS["year"], COLS["quarter"]],
    AGG["columns"],
    AGG["funcs"],
    nec_trn_arrivals,
    nec_detrn,
)
nec_qtr_stats

#### 3.4.1 Write to file [1 pt]

Write `nec_qtr_stats` to a CSV file named `stu-amtk-nec_qtr_stats.csv`. Store the file in the
`data/student` directory. Then compare it to the accompanying `fxt-amtk-nec_qtr_stats.csv` file.
It must match line for line, character for character.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
#hidden tests are within this cell

### 3.5 NEC: visualize detraining passengers

Visualize Amtrak's Northeast Corridor (NEC) detraining passengers, both on-time and late, across
all years and quarters with a bar chart.

In [ ]:
# Assemble the data for the chart
chrt_data = bar.create_detrain_chart_frame(nec_qtr_stats, CHRT_BAR["columns"])

# Create chart title
title_text = f"Amtrak {SVC_LINES['nec']} (NEC) Detraining Passengers"
title = ttl.format_title(nec_stats, title_text)

# Grouped bar chart
chart = bar.create_grouped_bar_chart(
    chrt_data,
    "Fiscal Period:N",
    "Passengers:Q",
    "Arrival Status:N",
    CHRT_BAR["xoffset_sort"],
    CHRT_BAR["colors"],
    title,
)

chart.display()

### 3.6 NEC: visualize distribution of mean late arrival times (by fiscal year and quarter)

Visualizing mean late arrival times grouped by fiscal year and quarter may reveal interesting
patterns.

The data is flattened prior to creating a series of box plots. The fiscal year and quarter columns
are combined (e.g., `< year >Q< quarter >`) to create a new column named "Fiscal Year Quarter" while
a second column is added to color code each quarter and its associated box plot.

#### 3.6.1 Create the chart data

In [ ]:
cols = [COLS["year"], COLS["quarter"], COLS["late_detrn_avg_mm_late"]]

# Group by fiscal year and quarter, flatten, and reset index
chrt_data = nec.groupby(cols[:2])[cols].apply(lambda x: x).reset_index(drop=True)

# Add column
chrt_data.loc[:, COLS["year_quarter"]] = chrt_data.apply(detrn.format_year_quarter, axis=1)

# Drop columns and reorder
chrt_data.drop(cols[:2], axis=1, inplace=True)
chrt_data.insert(0, COLS["year_quarter"], chrt_data.pop(COLS["year_quarter"]))

# Add alternating colors
colors = [COLORS["amtk_blue"], COLORS["amtk_red"]]
chrt_data.loc[:, "Color"] = chrt_data[COLS["year_quarter"]].apply(detrn.assign_color, colors=None)
chrt_data.head()

#### 3.6.2 Preaggregate the chart data

Attempting to instantiate an instance of a Vega-Altair [`alt.Chart()`](https://altair-viz.github.io/user_guide/generated/toplevel/altair.Chart.html) class by passing to it a dataset comprising more than `5000` rows will trigger a `MaxRowsError`. You can disable the `MaxRows` check by calling `alt.data_transformers.disable_max_rows()` method. However, disabling the check may result in performance issues, including browser crashes.

The preferred approach when [working with large datasets](https://altair-viz.github.io/user_guide/large_datasets.html#large-datasets) is to _preaggregate_ the data before generating a plot. This can be achieved "manually"&mdash;the approach adopted in this notebook&mdash;or by [installing](https://altair-viz.github.io/user_guide/large_datasets.html#installing-vegafusion[) and [enabling](https://altair-viz.github.io/user_guide/large_datasets.html#enabling-the-vegafusion-data-transformer) Altair's companion [vegafusion](https://vegafusion.io/) data transformer package.

In [ ]:
# Compute aggregation statistics
cols = [COLS["year_quarter"], COLS["late_detrn_avg_mm_late"]]

# Pre-aggregate the data
chrt_data = frm.aggregate_data(chrt_data, cols)

#### 3.6.3 Generate the box plots

In [ ]:
# Create chart title
title_text = f"Amtrak {SVC_LINES['nec']} (NEC) Late Detraining Passengers"
title = ttl.format_title(nec_stats, title_text)

chart_horizontal = boxp.create_boxplot(
    data=chrt_data,
    x_shorthand="Late Detraining Customers Avg Min Late:Q",
    x_title="Average Minutes Late",
    y_shorthand="Fiscal Year Quarter:N",
    y_title="Period",
    box_size=20,
    outlier_shorthand="outliers:Q",
    color_shorthand="Color:N",
    chart_title=title,
    orient=boxp.Orient.HORIZONTAL,
    height=400,
    width=680,
)
chart_horizontal.display()

## 4.0 State Supported services

In [ ]:
state = ntwk.by_service_line(network, SVC_LINES["state"])
state.shape

### 4.1 State Supported: on-time performance metrics (entire period) [1 pt]

State supported station performance data is a compilation of quarterly metrics that focus on late
detraining passengers. Detraining assengers are considered on-time if they arrive at their
destination no later than fifteen (`15`) minutes after their scheduled arrival time. All other
detraining passengers are considered late.

In [ ]:
# Total train arrivals
state_trn_arrivals = state.shape[1]

# Detraining totals
state_detrn = state[COLS["total_detrn"]][1:].sum()
state_detrn_late = state[COLS["late_detrn"]][1:].sum()
state_detrn_on_time = state_detrn - state_detrn_late

print(
    f"Train Arrivals: {state_trn_arrivals}",
    f"Total Detraining Customers: {state_detrn}",
    f"Late Detraining Customers: {state_detrn_late}",
    f"On-Time Detraining Customers: {state_detrn_on_time}",
    sep="\n",
)

# Compute summary statistics
state_stats = detrn.get_sum_stats(state, AGG["columns"], AGG["funcs"])
state_stats

In [ ]:
#hidden tests are within this cell

### 4.2 State Supported: mean late arrival times summary statistics

Review the central tendency, dispersion, and shape for the mean late arrival times of state
supported trains. Call the custom function named `frm.describe_numeric_column()` to return a
dictionary of summary statistics.

In [ ]:
# Drop missing values
state_avg_mm_late = state[COLS["late_detrn_avg_mm_late"]].dropna().reset_index(drop=True)

# Call the custom frm.describe_numeric_column() function again
state_avg_mm_late_describe = frm.describe_numeric_column(state_avg_mm_late)
state_avg_mm_late_describe

The skewness and kurtosis values returned suggest that the distribution of mean late arrival times
of NEC trains are positively skewed and features features a sharper peak and heavier right tail
than a normal distribution. Let's confirm this visually by generating a histogram.

### 4.3 State Supported: visualize distribution of mean late arrival times

Visualize mean late arrival times for the entire period. The data is binned prior to plotting.

#### 4.3.1 Create the chart data

In [ ]:
# Convert to DataFrame
state_avg_mm_late = state_avg_mm_late.to_frame(name=COLS["avg_mm_late"])

# Get mean and standard deviation
mu = state_avg_mm_late_describe["center"]["mean"]
sigma = state_avg_mm_late_describe["spread"]["std"]

# Get max value (for x-axis ticks); pad max value for chart display
max_val = state_avg_mm_late_describe["position"]["min"]
max_val_ceil = (np.ceil(max_val / 10) * 10).astype(int)

# Create bins
state_min_late, bins, num_bins, bin_width = frm.create_bins(
    state_avg_mm_late, COLS["avg_mm_late"], 15
)

# Bin the data
chrt_data = frm.bin_data(state_min_late, COLS["avg_mm_late"], bins)
# chrt_data

#### 4.3.2 Generate the histogram [1 pt]

In [ ]:
# Chart title
title_txt = f"Amtrak {SVC_LINES['state']} Late Detraining Passengers"
title = ttl.format_title(state_stats, title_txt)

# Tooltips
tooltip_config = [
    {"shorthand": "bin_center:Q", "title": "Average Minutes Late", "format": None},
    {"shorthand": "count:Q", "title": "Late Arrivals Count", "format": None},
]

# Create and display the histogram
chart = hst.create_histogram(
    frame=chrt_data,
    x_shorthand="bin_center:Q",
    x_title="Average Minutes Late",
    y_shorthand="count:Q",
    y_title="Late Arrivals Count",
    y_stack=False,
    line_shorthand="Avg Min Late:Q",
    mu=mu,
    sigma=sigma,
    num_bins=num_bins,
    bin_width=bin_width,
    x_tick_count_max=max_val_ceil,
    bar_color=COLORS["amtk_blue"],
    mu_color=COLORS["amtk_red"],
    sigma_color=COLORS["anth_gray"],
    tooltip_config=tooltip_config,
    title=title,
    width=680,
)
chart.display()


In [ ]:
#hidden tests are within this cell

### 4.4 State Supported: on-time performance metrics (by fiscal year and quarter) [1 pt]

Compute OTP summary statistics per fiscal year and quarter. Add quarterly train arrival metrics to
the `DataFrame` named `state_qtr_stats`.

In [ ]:
# Get quarterly stats
state_qtr_stats = detrn.get_sum_stats_by_group(
    nec,
    [COLS["year"], COLS["quarter"]],
    AGG["columns"],
    AGG["funcs"],
    state_trn_arrivals,
    state_detrn,
)
state_qtr_stats

In [ ]:
#hidden tests are within this cell

#### 4.4.1 Write to file [1 pt]

Write `state_qtr_stats` to a CSV file named `stu-amtk-state_qtr_stats.csv`. Store the file in the
`data/student` directory. Then compare it to the accompanying `fxt-amtk-state_qtr_stats.csv` file.
It must match line for line, character for character.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
#hidden tests are within this cell

### 4.5 State Supported: visualize detraining passengers

Visualize Amtrak's state-supported detraining passengers, both on-time and late, across all years
and quarters with a bar chart.

In [ ]:
# Assemble the data for the chart
chrt_data = bar.create_detrain_chart_frame(state_qtr_stats, CHRT_BAR["columns"])

# Create chart title
title_text = f"Amtrak {SVC_LINES['state']} Detraining Passengers"
title = ttl.format_title(state_stats, title_text)

# Grouped bar chart
chart = bar.create_grouped_bar_chart(
    chrt_data,
    "Fiscal Period:N",
    "Passengers:Q",
    "Arrival Status:N",
    CHRT_BAR["xoffset_sort"],
    CHRT_BAR["colors"],
    title,
)

chart.display()

### 4.6 State Supported: distribution of mean late arrival times (by fiscal year and quarter)

Visualizing mean late arrival times grouped by fiscal year and quarter may reveal interesting
patterns.

The data is flattened prior to creating a series of box plots. The fiscal year and quarter columns
are combined (e.g., `< year >Q< quarter >`) to create a new column named "Fiscal Year Quarter" while
a second column is added to color code each quarter and its associated box plot.

#### 4.6.1 Create the chart data [1 pt]

In [ ]:
cols = [COLS["year"], COLS["quarter"], COLS["late_detrn_avg_mm_late"]]

# Group by fiscal year and quarter, flatten, and reset index
state_avg_mm_late = state.groupby(cols[:2])[cols]

# Add column
state_avg_mm_late.loc[:, COLS["year_quarter"]] = state_avg_mm_late.apply(
    detrn.format_year_quarter, axis=1
)

# Drop columns and reorder
state_avg_mm_late.drop(cols[:2], axis=1, inplace=True)
state_avg_mm_late.insert(0, COLS["year_quarter"], state_avg_mm_late.pop(COLS["year_quarter"]))

# Add alternating colors
colors = [COLORS["amtk_blue"], COLORS["amtk_red"]]
state_avg_mm_late.loc[:, "Color"] = state_avg_mm_late[COLS["year_quarter"]].apply(
    detrn.assign_color, colors=colors
)

state_avg_mm_late

In [ ]:
#hidden tests are within this cell

#### 4.6.2 Preaggregate the data

In [ ]:
# Compute aggregation statistics
cols = [COLS["year_quarter"], COLS["late_detrn_avg_mm_late"]]

# Pre-aggregate the data
agg_stats = frm.aggregate_data(state_avg_mm_late, cols)

#### 4.6.3 Generate the box plots

In [ ]:
# Create chart title
title_text = f"Amtrak {SVC_LINES['state']} Late Detraining Passengers"
title = ttl.format_title(nec_stats, title_text)

chart_horizontal = boxp.create_boxplot(
    data=agg_stats,
    x_shorthand="Late Detraining Customers Avg Min Late:Q",
    x_title="Average Minutes Late",
    y_shorthand="Fiscal Year Quarter:N",
    y_title="Period",
    box_size=20,
    outlier_shorthand="outliers:Q",
    color_shorthand="Color:N",
    chart_title=title,
    orient=boxp.Orient.HORIZONTAL,
    height=400,
    width=680,
)
chart_horizontal.display()

## 5.0 Long Distance services

In [ ]:
long_dist = ntwk.by_service_line(network, SVC_LINES["long_dist"])
long_dist.shape

### 5.1 Long Distance: on-time performance metrics (entire period)

State supported station performance data is a compilation of quarterly metrics that focus on late
detraining passengers. Detraining assengers are considered on-time if they arrive at their
destination no later than fifteen (`15`) minutes after their scheduled arrival time. All other
detraining passengers are considered late.

In [ ]:
# Total train arrivals
long_dist_trn_arrivals = long_dist.shape[0]

# Detraining totals
long_dist_detrn = long_dist[COLS["total_detrn"]].sum()
long_dist_detrn_late = long_dist[COLS["late_detrn"]].sum()
long_dist_detrn_on_time = long_dist_detrn - long_dist_detrn_late

print(
    f"Train Arrivals: {long_dist_trn_arrivals}",
    f"Total Detraining Customers: {long_dist_detrn}",
    f"Late Detraining Customers: {long_dist_detrn_late}",
    f"On-Time Detraining Customers: {long_dist_detrn_on_time}",
    sep="\n",
)

# Compute summary statistics
long_dist_stats = detrn.get_sum_stats(long_dist, AGG["funcs"], AGG["columns"])
long_dist_stats

### 5.2 Long Distance: mean late arrival times summary statistics

Review the central tendency, dispersion, and shape for the mean late arrival times of long distance
trains. Call the custom function named `frm.describe_numeric_column()` to return a dictionary of
summary statistics.

In [ ]:
# Drop missing values
long_dist_avg_min_late = long_dist[COLS["late_detrn_avg_mm_late"]].dropna().reset_index(drop=True)

# Call the custom frm.describe_numeric_column() function again
long_dist_avg_min_late_describe = frm.describe_numeric_column(long_dist_avg_min_late)
long_dist_avg_min_late_describe

The skewness and kurtosis values returned suggest that the distribution of mean late arrival times
of long distance trains are positively skewed and features features a sharper peak and heavier right
tail than a normal distribution. Let's confirm this visually by generating a histogram.

### 5.3 Long Distance: visualize detraining passengers

Visualize mean late arrival times for the entire period. The data is binned prior to plotting.

#### 5.3.1 Create the chart data

In [ ]:
# Convert to DataFrame
long_dist_avg_min_late = long_dist_avg_min_late.to_frame(name=COLS["avg_mm_late"])

# Get mean and standard deviation
mu = long_dist_avg_min_late_describe["center"]["mean"]
sigma = long_dist_avg_min_late_describe["spread"]["std"]

# Get max value (for x-axis ticks); pad max value for chart display
max_val = long_dist_avg_min_late_describe["position"]["max"]
max_val_ceil = (np.ceil(max_val / 10) * 10).astype(int)

# Create bins
long_dist_min_late, bins, num_bins, bin_width = frm.create_bins(
    long_dist_avg_min_late, COLS["avg_mm_late"], 10
)

# Bin the data
chrt_data = frm.bin_data(long_dist_min_late, COLS["avg_mm_late"], bins)
# chrt_data

#### 5.3.2 Generate the histogram

In [ ]:
# Chart title
title_txt = f"Amtrak {SVC_LINES['long_dist']} Service Late Detraining Passengers"
title = ttl.format_title(long_dist_stats, title_txt)

# Tooltips
tooltip_config = [
    {"shorthand": "bin_center:Q", "title": "Average Minutes Late", "format": None},
    {"shorthand": "count:Q", "title": "Late Arrivals Count", "format": None},
]

# Create and display the histogram
chart = hst.create_histogram(
    frame=chrt_data,
    x_shorthand="bin_center:Q",
    x_title="Average Minutes Late",
    y_shorthand="count:Q",
    y_title="Late Arrivals Count",
    y_stack=False,
    line_shorthand="Avg Min Late:Q",
    mu=mu,
    sigma=sigma,
    num_bins=num_bins,
    bin_width=bin_width,
    x_tick_count_max=max_val_ceil,
    bar_color=COLORS["amtk_blue"],
    mu_color=COLORS["amtk_red"],
    sigma_color=COLORS["anth_gray"],
    tooltip_config=tooltip_config,
    title=title,
    width=680,
)
chart.display()

### 5.4 Long Distance: on-time performance metrics (by fiscal year and quarter)

Compute OTP summary statistics per fiscal year and quarter. Add quarterly train arrival metrics to
the `DataFrame` named `long_dist_qtr_stats`.

In [ ]:
# Get quarterly stats
long_dist_qtr_stats = detrn.get_summary_stats_by_group(
    long_dist,
    [COLS["year"], COLS["quarter"]],
    AGG["columns"],
    AGG["funcs"],
    long_dist_trn_arrivals,
    long_dist_detrn,
)
long_dist_qtr_stats

#### 5.4.1 Write to file [1 pt]

Write `long_dist_qtr_stats` to a CSV file named `stu-amtk-long_dist_qtr_stats.csv`. Store the file in the
`data/student` directory. Then compare it to the accompanying `fxt-amtk-long_dist_qtr_stats.csv` file.
It must match line for line, character for character.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
#hidden tests are within this cell

### 5.5 Long Distance: visualize detraining passengers

Visualize Amtrak's long distance detraining passengers, both on-time and late, across all years
and quarters with a bar chart.

In [ ]:
# Assemble the data for the chart
chrt_data = bar.create_detrain_chart_frame(long_dist_qtr_stats, CHRT_BAR["columns"])

# Create chart title
title_text = f"Amtrak {SVC_LINES['long_dist']} Detraining Passengers"
title = ttl.format_title(long_dist_stats, title_text)

# Grouped bar chart
chart = bar.create_grouped_bar_chart(
    chrt_data,
    "Fiscal Period:N",
    "Passengers:Q",
    "Arrival Status:N",
    CHRT_BAR["xoffset_sort"],
    CHRT_BAR["colors"],
    title,
)

chart.display()

###  5.6 Long Distance: distribution of mean late arrival times (by fiscal year and quarter)

Visualizing mean late arrival times grouped by fiscal year and quarter may reveal interesting
patterns.

The data is flattened prior to creating a series of box plots. The fiscal year and quarter columns
are combined (e.g., `< year >Q< quarter >`) to create a new column named "Fiscal Year Quarter" while
a second column is added to color code each quarter and its associated box plot.

#### 5.6.1 Create the chart data

In [ ]:
cols = [COLS["year"], COLS["quarter"], COLS["late_detrn_avg_mm_late"]]

# Group by fiscal year and quarter, flatten, and reset index
long_dist_avg_min_late = long_dist.groupby(cols[:2])[cols].apply(lambda x: x).reset_index(drop=True)

# Add column
long_dist_avg_min_late.loc[:, COLS["year_quarter"]] = long_dist_avg_min_late.apply(
    detrn.format_year_quarter, axis=1
)

# Drop columns and reorder
long_dist_avg_min_late.drop(cols[:2], axis=1, inplace=True)
long_dist_avg_min_late.insert(
    0, COLS["year_quarter"], long_dist_avg_min_late.pop(COLS["year_quarter"])
)

# Add alternating colors
colors = [COLORS["amtk_blue"], COLORS["amtk_red"]]
long_dist_avg_min_late.loc[:, "Color"] = long_dist_avg_min_late[COLS["year_quarter"]].apply(
    detrn.assign_color, colors=colors
)
long_dist_avg_min_late

#### 5.6.2 Preaggregate the data

In [ ]:
# Compute aggregation statistics
cols = [COLS["late_detrn_avg_mm_late"], COLS["year_quarter"]]

# Pre-aggregate the data
chrt_data = frm.aggregate_data(long_dist_avg_min_late, cols)

#### 5.6.3 Generate box plots [1 pt]

In [ ]:
# Create chart title
title_text = f"Amtrak {SVC_LINES['state']} Late Detraining Passengers"
title = ttl.format_title(nec_stats, title_text)

chart_horizontal = boxp.create_boxplot(
    data=chrt_data,
    x_shorthand="Late Detraining Customers Avg Min Late:Q",
    x_title="Average Minutes Late",
    y_shorthand="Fiscal Year Quarter:N",
    y_title="Period",
    box_size=20,
    outlier_shorthand="outliers:Q",
    color_shorthand="Color:N",
    chart_title=title,
    orient=boxp.Orient.HORIZONTAL,
    height=400,
    width=680,
)
chart_horizontal.display()

In [ ]:
#hidden tests are within this cell

## 6.0 Watermark

In [ ]:
%load_ext watermark
%watermark -h -i -iv -m -v